In [ ]:
#@title Install Dependancies
!pip install -q keras-bert keras-rectified-adam
%tensorflow_version 1.x
!pip install -q scikit-plot

TensorFlow 1.x selected.


In [ ]:
#@title Set Evnironment
import os
import contextlib
import codecs
import numpy as np
from tqdm import tqdm
import scikitplot as skplt
from sklearn.metrics import accuracy_score, f1_score

os.environ['TF_KERAS'] = '1'

# Tensorflow Imports
import tensorflow as tf
from tensorflow.python import keras
import tensorflow.keras.backend as K

# Keras-bert imports
from keras_radam import RAdam
from keras_bert import Tokenizer
from keras_bert import get_custom_objects
from keras_bert import load_trained_model_from_checkpoint

# Bert Model Constants
SEQ_LEN = 128
BATCH_SIZE = 16
EPOCHS = 3
LR = 2e-5

min_exp = 24
max_exp = 25


pretrained_path = 'uncased_L-12_H-768_A-12'
config_path = os.path.join(pretrained_path, 'bert_config.json')
checkpoint_path = os.path.join(pretrained_path, 'bert_model.ckpt')
vocab_path = os.path.join(pretrained_path, 'vocab.txt')

tf.debugging.set_log_device_placement(True)

In [ ]:
# @title Download Bert Model
!wget -q https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
!unzip -o uncased_L-12_H-768_A-12.zip


Archive:  uncased_L-12_H-768_A-12.zip
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: uncased_L-12_H-768_A-12/vocab.txt  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: uncased_L-12_H-768_A-12/bert_config.json  


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# @title Prepare training and test data

token_dict = {}
with codecs.open(vocab_path, 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        token_dict[token] = len(token_dict)

tokenizer = Tokenizer(token_dict)

def load_data(path, tagset):
    """
    Input:
      path: Root directory where the categorical data sit in folders.
      tagset: List of folder-name, category tuples.
    Output:
      train_x / test_x: List with two items (viz token_input, seg_input)
      test_x / test_y: Output labels corresponding to trainx / train_y.
    """
    global tokenizer
    items = []
    
    # Read positive examples
    indices, sentiments = [], []
    fpos = open(os.path.join(path, 'pos.csv'))
    for line in fpos:
      ids, segments = tokenizer.encode(line, max_len=SEQ_LEN)
      indices.append(ids)
      sentiments.append(1)
    items += list(zip(indices, sentiments))
    
    # Read negative examples
    indices, sentiments = [], []
    fneg = open(os.path.join(path, 'neg.csv'))
    for line in fneg:
      ids, segments = tokenizer.encode(line, max_len=SEQ_LEN)
      indices.append(ids)
      sentiments.append(0)
    items += list(zip(indices, sentiments))

    np.random.shuffle(items)
    indices, sentiments = zip(*items)
    indices = np.array(indices)
    mod = indices.shape[0] % BATCH_SIZE
    if mod > 0:
        indices, sentiments = indices[:-mod], sentiments[:-mod]
    return [indices, np.zeros_like(indices)], np.array(sentiments)
  
  
#train_path = os.path.join(os.path.dirname(dataset), 'data', 'train')
#test_path = os.path.join(os.path.dirname(dataset), 'data', 'test')


tagset = [('neg', 0), ('pos', 1)]
id_to_labels = {0: 'negative', 1: 'positive'}


In [ ]:
with open('test_results.csv', 'a') as test_res_file:
  test_res_file.write('RUN_NO,ACCURACY,F1SCORE,CMATVALS' + '\n')

with open('train_results_acc.csv', 'a') as train_res_file:
  train_res_file.write( 'RUN_NO,' + ','.join(['ACCURACY_' + str(i) for i in range(1, 3 + 1)]) + '\n')

with open('train_results_loss.csv', 'a') as train_res_file:
  train_res_file.write( 'RUN_NO,' + ','.join(['LOSS_' + str(i) for i in range(1, 3 + 1)]) + '\n')


In [ ]:
# @title Load Bert and add SD Layer

for run_no in range(min_exp, max_exp + 1):
  test_path = os.path.join('drive/MyDrive/experiments' , 'run_' + str(run_no) , 'test')
  train_path = os.path.join('drive/MyDrive/experiments' , 'run_' + str(run_no) , 'train')

  train_x, train_y = load_data(train_path, tagset)
  test_x, test_y = load_data(test_path, tagset)
  # Load pretrained model
  with strategy.scope() if False else contextlib.suppress():
    model = load_trained_model_from_checkpoint(
        config_path,
        checkpoint_path,
        training=True,
        trainable=True,
        seq_len=SEQ_LEN,
    )
    inputs = model.inputs[:2]
    dense = model.get_layer('NSP-Dense').output
    outputs = keras.layers.Dense(units=2, activation='softmax')(dense)
    model = keras.models.Model(inputs, outputs)

    model.compile(
        RAdam(lr=LR),
        loss='sparse_categorical_crossentropy',
        metrics=['sparse_categorical_accuracy'],
    )
  sess = K.get_session()
  uninitialized_variables = set([i.decode('ascii') for i in sess.run(tf.report_uninitialized_variables())])
  init_op = tf.variables_initializer(
      [v for v in tf.global_variables() if v.name.split(':')[0] in uninitialized_variables]
  )
  sess.run(init_op)

  history = model.fit(
    train_x,
    train_y,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_split=0.20,
    shuffle=True,
  )

  with open('train_results_acc.csv', 'a') as train_res_file:
    train_res_file.write( 'run' + str(run_no) + ',' + ','.join( [str(ba) for ba in history.history['sparse_categorical_accuracy']]) + '\n')

  with open('train_results_loss.csv', 'a') as train_res_file:
    train_res_file.write( 'run' + str(run_no) + ',' + ','.join( [str(ba) for ba in history.history['loss']]) + '\n')

  #@subtitle Run model and log training / test data
  predicts = model.predict(test_x, verbose=True).argmax(axis=-1)

  with open('test_results.csv', 'a') as test_res_file:
    accuracy = accuracy_score(test_y, predicts)
    micro_f1 = f1_score(test_y, predicts, average='micro')
    cmat_vals = skplt.metrics.confusion_matrix(
                    [id_to_labels[x] for x in test_y], 
                    [id_to_labels[x] for x in predicts])
    #cmat_vals = [[id_to_labels[x] for x in test_y], [id_to_labels[x] for x in predicts]]
    test_res_file.write('run' + str(run_no) + ',' + str(accuracy) + ',' + str(micro_f1) + ',' + str(cmat_vals).replace('\n', '') + '\n')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 9600 samples, validate on 2400 samples
Epoch 1/3
9600/9600 [==============================] - 225s 23ms/sample - loss: 0.3891 - sparse_categorical_accuracy: 0.8205 - val_loss: 0.3110 - val_sparse_categorical_accuracy: 0.8729
Epoch 2/3
9600/9600 [==============================] - 207s 22ms/sample - loss: 0.2096 - sparse_categorical_accuracy: 0.9210 - val_loss: 0.2696 - val_sparse_categorical_

In [ ]:
from shutil import copyfile
copyfile("train_results_acc.csv", "drive/MyDrive/train_results_acc.csv")
copyfile("train_results_loss.csv", "drive/MyDrive/train_results_loss.csv")
copyfile("test_results.csv", "drive/MyDrive/test_results.csv")

In [ ]:
#@title download results
from google.colab import files
files.download("train_results_acc.csv")
files.download("train_results_loss.csv")
files.download("test_results.csv")